In [285]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import cufflinks as cf
cf.go_offline()

# Scipy
from scipy.spatial.distance import pdist, squareform

#sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel

#Librería Spotify
import spotipy
#from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util 


In [286]:
def connect_spotify(client_id, client_secret, AUTH_URL):
    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    # convert the response to JSON
    auth_response_data = auth_response.json()
    # save the access token
    access_token = auth_response_data['access_token']
    return access_token

In [287]:
client_id = 
client_secret = 
AUTH_URL = 
redirect_uri=
username=
access_token = connect_spotify(client_id, client_secret, AUTH_URL)



In [288]:
#Acces to modify the playlist
scope="playlist-modify-public"

token=util.prompt_for_user_token(username,scope,client_id="",client_secret="",redirect_uri="http://localhost:8080")
sp = spotipy.Spotify(auth=token)

In [289]:
#Function to obtain the features of a song.
def song_features(track):
    headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}
    BASE_URL = 'https://api.spotify.com/v1/'
    track_id = track
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    response_features=r.json()
    return pd.DataFrame([response_features])

In [290]:
#Function to get the id's songs. 
def get_id_songs(playlist):    
    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    BASE_URL = 'https://api.spotify.com/v1/'
    r = requests.get(BASE_URL + 'playlists/'+playlist+'/', headers=headers)
    response_tracks=r.json()
    lst=[]
    lst = [i['track']['id'] for i in response_tracks['tracks']['items']]
    return(lst)

In [291]:
#Function to get the features of the songs from a playlist.
def playlist_features(playlist):
    songs = get_id_songs(playlist)
    df_features = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
        'time_signature'])
    for i in songs:
        track=i
        features = song_features(track)
        total=[df_features, features]
        df_features=pd.concat(total)
    return df_features

In [292]:
#Function to get the features of the songs from database.
def all_playlist_features():
    songs = get_id_songs(playlist)
    global track 
    while track in songs: 
        songs.remove(track)
    df_features = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
        'time_signature'])
    for i in songs:
        track=i
        features = song_features(track)
        total=[df_features, features]
        df_features=pd.concat(total)
    return df_features

New playlist with recommendations


In [293]:
playlist='2VSX8asoMmFJkOfybpgLsz'
df_features= playlist_features(playlist)
df_features.drop(['type','uri','track_href','analysis_url','duration_ms'], axis = 'columns', inplace=True)
df_features.set_index('id', inplace = True)
df_features=df_features.astype('float64')
df_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
id,,,,,,,,,,,,
0RYDPGXliFYXfRgs4mj7Ka,0.598,0.399,1.0,-7.142,1.0,0.0593,0.79,0.000003,0.109,0.699,203.94,4.0


In [294]:
data_songs=pd.read_csv('datasets/data_songs_clean.csv').set_index('id', drop=True)

In [295]:
#Eliminar las canciones elegidas como muestra del df global.
for song in df_features.index:
    data_songs=data_songs.drop(index=song)

In [296]:
#Concateno

frames = [data_songs, df_features]

#concatenate dataframes
df_mix = pd.concat(frames, sort=False)

In [297]:
#Elimino los nulos.
df_mix.dropna(inplace=True)

In [298]:
#Extraigo solo las features
df_mix_values= df_mix.iloc[:,1:].values

In [299]:
#Extandarizo los df. Media 0 y desviación típica 1
scaler = StandardScaler()
df_mix_scaled = scaler.fit_transform(df_mix_values)

In [300]:
#Normalización:

#Calculo la magnitud del Vector para normalizar el vector.
data_songs_mag = np.sqrt((df_mix_scaled*df_mix_scaled).sum(axis=1))

ndata_song=df_mix_scaled.shape[0]
normalized_data_songs=df_mix_scaled/data_songs_mag.reshape(ndata_song,1)

In [301]:
#normalized_data_songs_df = pd.DataFrame(normalized_data_songs)
#normalized_data_songs_df.dropna(inplace=True)
#normalized_data_songs = normalized_data_songs_df.to_numpy()

In [302]:
cos_sim = linear_kernel(normalized_data_songs,normalized_data_songs[-1:])
cos_sim.shape

(6958, 1)

In [303]:
print(cos_sim[2,0])

0.44930523336680017


In [304]:
cos_sim_df = pd.DataFrame(cos_sim)

In [305]:
songs_ponderate = pd.concat([df_mix.reset_index(), cos_sim_df], axis=1)

In [306]:
songs_ponderate.columns

Index([              'id',     'danceability',           'energy',
                    'key',         'loudness',             'mode',
            'speechiness',     'acousticness', 'instrumentalness',
               'liveness',          'valence',            'tempo',
         'time_signature',                  0],
      dtype='object')

In [307]:
songs_for_0 = songs_ponderate.sort_values(by=[0],ascending=False)

songs_for_0 

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,0
6957,0RYDPGXliFYXfRgs4mj7Ka,0.598,0.399,1.0,-7.142,1.0,0.0593,0.79000,0.000003,0.1090,0.699,203.940,4.0,1.000000
2646,50uGbeaQIxKiSc7jvRTjWx,0.484,0.312,2.0,-13.522,1.0,0.0535,0.72400,0.000002,0.0601,0.598,205.981,4.0,0.952150
3955,08cXy6KUizaAelYXtcew3w,0.388,0.595,1.0,-9.799,1.0,0.0325,0.66300,0.000056,0.1780,0.596,190.043,4.0,0.950702
2906,6XAcmxPbCfMzLBvjBd2G0H,0.430,0.558,1.0,-6.549,1.0,0.0844,0.53400,0.000507,0.0914,0.779,172.756,4.0,0.943347
5352,4SipDsIDlvpv37P41D7Ec3,0.640,0.612,0.0,-7.273,1.0,0.0370,0.67500,0.000357,0.0655,0.761,178.715,4.0,0.939466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,7ajdfpfW1h6cN7GMQ5Sver,0.641,0.730,7.0,-6.406,0.0,0.0650,0.01030,0.145000,0.4120,0.605,75.001,4.0,-0.754907
6685,4dF3KsznC2QkvlO54A7jr0,0.449,0.806,7.0,-4.738,0.0,0.0384,0.00336,0.000000,0.2030,0.326,77.504,4.0,-0.784846
4276,1vwUsp52io0AGQ5yv470IC,0.387,0.640,11.0,-5.817,0.0,0.0810,0.10700,0.000000,0.3190,0.617,59.989,4.0,-0.787356
4472,3jbT1Y5MoPwEIpZndDDwVq,0.734,0.836,10.0,-4.803,0.0,0.0735,0.01700,0.000016,0.1790,0.623,80.002,4.0,-0.787841


In [308]:
#songs_for_1 = songs_ponderate.sort_values(by=[1],ascending=False)

#songs_for_1 

In [309]:
best_tracks=songs_for_0['id'].head(50)

In [310]:
tracks = best_tracks.values.tolist()

In [311]:
tracks=["spotify:track:" + a for a in tracks]

In [312]:
sp.playlist_add_items(playlist_id=playlist, items=tracks, position=None)

{'snapshot_id': 'Myw3NTQ1OWUzNWE3ZmRkNWJmYWY3Yzk1YjNjNTQwODg5YWU2ZWNlYjNi'}